Without coinstrints 19-21  
Without S

In [1]:
import pandas as pd
import numpy as np
import itertools
import os
import collections
from pulp import *

In [2]:
minimise = True

In [3]:
if minimise:
    problem = LpProblem("E-Scooter Allocation Carbon Minimisation", LpMinimize)
else:
    problem = LpProblem("E-Scooter Allocation Profit Maximisation", LpMaximize)

In [4]:
df_distance = pd.read_csv("Data/Model Data - Smaller set Distance.csv")
df_distance.fillna(0, inplace=True)
distance = df_distance.values[:,1:]
distance_dict = { (x,y): distance[x][y] for x in range(distance.shape[0]) for y in range(distance.shape[1])}

In [5]:
locations = list(df_distance['Location'])
location_idx = np.arange(0, len(locations))
loc_count = len(locations)

In [6]:
d = dict()
files = os.listdir('Data/Reduced set Demand/')
for f in files:
    hour = int(f.split(sep='.', maxsplit=1)[0])
    df = pd.read_csv('Data/Reduced set Demand/' + f)
    demand_arr = df.values[:,1:]
    d[hour] = demand_arr

In [7]:
demand = collections.OrderedDict(sorted(d.items())) 

In [8]:
demand_dict = dict()
for i,(_,v) in enumerate(demand.items()):
    demand_dict.update({(x,y,i): v[x][y] for x in range(loc_count) for y in range(loc_count)})

In [9]:
# Parameters
Nmax = 10
Zmax = 100
Zmin = 1
# Carbon Cost values
Cc_scooter_km = 7
Cc_fixed_scooter = 167
Cc_fixed_dock = 46
Cc_fixed_station = 6
# Cost relocation
Cc_r = 160

scalar = 1.25

Qmin = 0
# Cost values
# cost of maintaining one scooter per kilometer driven
Cp_scooter_km = 0.4
Cp_fixed_scooter = 0.38
Cp_fixed_dock = 0.92
Cp_fixed_station = 2.3
# Cost relocation
Cp_r = 1.6
# Price rate for kilometer driven
P_km = 0.6
# Price for pickup
P_init = 1

In [10]:
# Penalty
df_penalty = pd.read_csv("Data/Penalty Carbon Costs Smaller set.csv")
C_pen = df_penalty.values[:, 1]
C_pen = C_pen*scalar

In [11]:
# Sets
T = np.arange(0,len(files)+1)
# each location at a given time
X = list(itertools.product(location_idx, T))
#X2 = [(i,it) for (i,it) in X if sum([demand_dict[(i,j,it)] for j in location_idx if i !=j and it != T[-1]]) > 0]
A1 = [(xi, xj) for xi in X for xj in X if xi[0] != xj[0] and xi[1]+1==xj[1]]
# Relocation
A3 = [(xi, xj) for xi in X for xj in X if xi[0]!=xj[0] and xi[1]==T[-1] and xj[1]==T[0]]

In [12]:
# Decision Variables
Yi = LpVariable.dicts("Station Presence", location_idx,0,cat=const.LpBinary)
Zi = LpVariable.dicts("Size", location_idx, 0, cat="Continuous")
# Relocation
Rij = LpVariable.dicts("#Relocated_Scooters", A3, 0, cat=const.LpInteger)
R = LpVariable.dicts('Relocation needed', A3, 0, cat=const.LpBinary)
Vit = LpVariable.dicts("#Available_Scooters",X,0,cat="Continuous")
#Sit = LpVariable.dicts("#Stocked_Scooters", A2, 0, cat="Continuous")
Ditj = LpVariable.dicts("#Used_Scooters", A1, 0 ,cat="Continuous")
#Xit = LpVariable.dicts("Availability binary", X2, 0, cat=const.LpBinary)

if minimise:
    O = LpVariable("Profit Max Objective function value", 0, cat='Continuous')
else:
    O = LpVariable("Carbon Min Objective function value", 0, cat='Continuous')

In [13]:
# Objective function
if minimise:
    problem+=lpSum(Ditj[((i,ti),(j,tj))]*distance_dict[(i,j)]*Cc_scooter_km for ((i,ti),(j,tj)) in A1) + \
    lpSum((demand_dict[(i,j,ti)]-Ditj[((i,ti),(j,tj))])*distance_dict[(i,j)]*C_pen[i] for ((i,ti),(j,tj)) in A1) + \
    lpSum(R[((i,ti), (j,tj))]*distance_dict[(i,j)]*Cc_r for ((i,ti), (j,tj)) in A3) + \
    lpSum(Vit[(i,t)]*Cc_fixed_scooter for (i,t) in X if t==0) + \
    lpSum(Zi[i]*Cc_fixed_dock for i in location_idx) + \
    lpSum(Yi[i]*Cc_fixed_station for i in location_idx), "Objective function"

    problem+= O == lpSum(Ditj[((i,ti),(j,tj))]*(distance_dict[(i,j)]*(P_km-Cp_scooter_km) + P_init) for ((i,ti),(j,tj)) in A1) - \
    lpSum(R[((i,ti), (j,tj))]*distance_dict[(i,j)]*Cp_r for ((i,ti), (j,tj)) in A3) - \
    lpSum(Vit[(i,t)]*Cp_fixed_scooter for (i,t) in X if t==0) - \
    lpSum(Zi[i]*Cp_fixed_dock for i in location_idx) - \
    lpSum(Yi[i]*Cp_fixed_station for i in location_idx), "Profit Max Objective function value"
else:
    problem+=lpSum(Ditj[((i,ti),(j,tj))]*(distance_dict[(i,j)]*(P_km-Cp_scooter_km) + P_init) for ((i,ti),(j,tj)) in A1) - \
    lpSum(R[((i,ti), (j,tj))]*distance_dict[(i,j)]*Cp_r for ((i,ti), (j,tj)) in A3) - \
    lpSum(Vit[(i,t)]*Cp_fixed_scooter for (i,t) in X if t==0) - \
    lpSum(Zi[i]*Cp_fixed_dock for i in location_idx) - \
    lpSum(Yi[i]*Cp_fixed_station for i in location_idx), "Objective function"

    problem+= O == lpSum(Ditj[((i,ti),(j,tj))]*distance_dict[(i,j)]*Cc_scooter_km for ((i,ti),(j,tj)) in A1) + \
    lpSum((demand_dict[(i,j,ti)]-Ditj[((i,ti),(j,tj))])*distance_dict[(i,j)]*C_pen[i] for ((i,ti),(j,tj)) in A1) + \
    lpSum(R[((i,ti), (j,tj))]*distance_dict[(i,j)]*Cc_r for ((i,ti), (j,tj)) in A3) + \
    lpSum(Vit[(i,t)]*Cc_fixed_scooter for (i,t) in X if t==0) + \
    lpSum(Zi[i]*Cc_fixed_dock for i in location_idx) + \
    lpSum(Yi[i]*Cc_fixed_station for i in location_idx), "Carbon Min Objective function value"


In [14]:
for a3 in A3:
    problem+=Rij[a3]<=Zmax*R[a3], f"Relocation needed for {a3}"

In [15]:
for (i,ti) in X:
    if ti !=0:
        problem+= Vit[(i,ti-1)] - lpSum(Ditj[((i,ti-1),(j,ti))] for j in location_idx if i !=j) + lpSum(Ditj[((j, ti-1),(i, ti))] for j in location_idx if i !=j) == Vit[(i,ti)], f"Availability Balance {(i,ti)}"


In [16]:
for (i,ti) in X:
    if ti!=T[-1]:
        problem+=Vit[(i,ti)] - lpSum(Ditj[((i,ti),(j,ti+1))] for j in location_idx if i !=j) >=0 , f"Stocked Scooters for {(i,ti)}"

In [17]:
# Relocation constraint
for i in location_idx:
    problem+=Vit[(i,0)] == Vit[(i,T[-1])] + lpSum(Rij[((j,T[-1]), (i,0))] for j in location_idx if i!=j) - lpSum(Rij[((i,T[-1]), (j,0))] for j in location_idx if i!=j), f"Relocation balance for location {i}"

In [18]:
for i in location_idx:
    problem+=lpSum(Rij[((i,T[-1]), (j,0))] for j in location_idx if i!=j) <= Vit[(i,T[-1])], f"Relocation availability for location {i}"

In [19]:
for (i,ti) in X:
    problem+= Zi[i] >=Vit[(i,ti)], f"Size constraint for {(i,ti)}"

In [20]:
for (i,ti) in X:
    problem+=Vit[(i,ti)] <= Zmax*Yi[i], f"Maximum size constraint {(i,ti)}"

In [21]:
for i in location_idx:
    problem+=Yi[i]*Zmin <=Zi[i], f"Minimum size constraint for {i}"

In [22]:
for ((i,ti),(j,tj)) in A1:
    problem+=Ditj[((i,ti),(j,tj))]<=demand_dict[(i,j,ti)], f"Maximum Demand for {((i,ti),(j,tj))}"

In [23]:
problem+=lpSum(Yi[i] for i in location_idx) <= Nmax, f"Maximum number of docks"

In [24]:
if not minimise:
    problem+=lpSum(v for v in Ditj.values())/lpSum(v for v in demand_dict.values()) >= Qmin, "Minimum satisfied demand"

In [25]:
# for (i,ti) in X2:
#     problem+= Xit[(i,ti)] <= (Vit[(i,ti)] + lpSum(demand_dict[(i,j,ti)] for j in location_idx if i!=j) - lpSum(demand_dict[(i,j,ti)]*Yi[j] for j in location_idx if i!=j))/lpSum(demand_dict[(i,j,ti)] for j in location_idx if i!=j), f"Xit value constraint for {(i,ti)}"

In [26]:
# for (i,ti) in X2:
#     if ti != T[-1]:
#         problem+=Vit[(i,ti)] - lpSum(Ditj[((i,ti),(j,ti+1))] for j in location_idx if i !=j) <= Zmax*Xit[(i,ti)]

In [27]:
# for (i,ti) in X2:
#     if ti != T[-1]:
#         problem+= lpSum(Ditj[((i,ti), (j,ti+1))] for j in location_idx if i!=j) >= lpSum(demand_dict[(i,j,ti)]*(Xit[(i,ti)]-1) for j in location_idx if i!=j) + lpSum(demand_dict[(i,j,ti)]*Yi[j] for j in location_idx if i!=j)

In [28]:
# if minimise:
#     problem.writeLP("E-scooterProblem Carbon Min.lp")
# else:
#     problem.writeLP("E-scooterProblem Profit Max.lp")
problem.solve()
LpStatus[problem.status]

'Optimal'

In [29]:
for v in problem.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

#Available_Scooters_(12,_0) = 36.0
#Available_Scooters_(12,_1) = 33.0
#Available_Scooters_(12,_10) = 3.0
#Available_Scooters_(12,_11) = 11.0
#Available_Scooters_(12,_12) = 19.0
#Available_Scooters_(12,_13) = 25.0
#Available_Scooters_(12,_14) = 33.0
#Available_Scooters_(12,_15) = 33.0
#Available_Scooters_(12,_16) = 34.0
#Available_Scooters_(12,_17) = 34.0
#Available_Scooters_(12,_18) = 35.0
#Available_Scooters_(12,_19) = 36.0
#Available_Scooters_(12,_2) = 24.0
#Available_Scooters_(12,_3) = 14.0
#Available_Scooters_(12,_4) = 6.0
#Available_Scooters_(12,_5) = 4.0
#Available_Scooters_(12,_6) = 3.0
#Available_Scooters_(12,_7) = 3.0
#Available_Scooters_(12,_8) = 2.0
#Available_Scooters_(12,_9) = 1.0
#Available_Scooters_(13,_0) = 10.0
#Available_Scooters_(13,_1) = 10.0
#Available_Scooters_(13,_10) = 3.0
#Available_Scooters_(13,_11) = 5.0
#Available_Scooters_(13,_12) = 8.0
#Available_Scooters_(13,_13) = 6.0
#Available_Scooters_(13,_14) = 8.0
#Available_Scooters_(13,_15) = 9.0
#Available_Scoote

In [30]:
value(problem.objective)

165819.7035162501

In [31]:
O.varValue

591.677

In [32]:
# Extracting the results
#(idx, LpVariable)
size_list = []
for _,v in Zi.items():
    size_list.append(v.value())  

In [33]:
station_list = []
for _,v in Yi.items():
    station_list.append(v.value()) 

In [34]:
df_results = pd.DataFrame(list(zip(locations, station_list, size_list)), columns=["Location","Station", "Size"])
df_results

,Location,Station,Size
0,Balgreen and Roseburn,0.0,0.0
1,Blackhall,0.0,0.0
2,Broughton North and Powderhall,1.0,12.0
3,Broughton South,0.0,0.0
4,Bruntsfield,1.0,5.0
5,Comely Bank,0.0,0.0
6,Dalry and Fountainbridge,1.0,21.0
7,Drylaw,0.0,0.0
8,Gorgie East,0.0,0.0
9,Hillside and Calton Hill,1.0,8.0


In [35]:
if minimise:
    df_results.to_csv("results-carbon-reduced.csv", index=False)
else:
    df_results.to_csv("results-profit-reduced.csv", index=False)

In [36]:
total_Vit = dict()
for k,v in Vit.items():
    if k[1] not in total_Vit:
        total_Vit[k[1]] = int(v.varValue)
    else:
        total_Vit[k[1]]+=int(v.varValue)

In [37]:
print("#Scooters: " + str(total_Vit[0]))
total_scooters = total_Vit[0]

#Scooters: 124


In [38]:
total_demand = 0
for _,v in demand_dict.items():
    total_demand+=v
total_demand

1813

In [39]:
satisfied_demand = 0
for _,v in Ditj.items():
    satisfied_demand+=v.varValue
satisfied_demand

679.0

In [40]:
satisfied_demand/total_demand

0.3745173745173745

In [41]:
trips = dict()
for ((i,ti),(j,tj)) in Ditj.keys():
    if ti not in trips.keys():
        trips[ti] = Ditj[((i,ti),(j,tj))].varValue
    else:
        trips[ti] +=Ditj[((i,ti),(j,tj))].varValue
trips

{0: 23.0,
 1: 54.0,
 2: 98.0,
 3: 62.0,
 4: 26.0,
 5: 14.0,
 6: 4.0,
 7: 7.0,
 8: 6.0,
 9: 43.0,
 10: 72.0,
 11: 76.0,
 12: 84.0,
 13: 61.0,
 14: 22.0,
 15: 11.0,
 16: 3.0,
 17: 5.0,
 18: 8.0}

In [42]:
satisfied_demand/total_scooters

5.475806451612903

In [43]:
# Hard coded values - not relevant at this point
Obj_matrix = np.array([[165819.70351625013,591.677],[177487.510000, 644.8944]])
df_obj = pd.DataFrame(Obj_matrix, columns=["Carbon", "Profit"])
df_obj["Obj"] = ["Carbon", "Profit"]

In [44]:
cols = df_obj.columns.tolist()
cols = cols[-1:] + cols[:-1]
df_obj = df_obj[cols]
df_obj

,Obj,Carbon,Profit
0,Carbon,165819.703516,591.6770
1,Profit,177487.510000,644.8944


In [45]:
for k in Rij.keys():
    print(f"Rij[{k}] = {Rij[k].varValue}")

Rij[((0, 19), (1, 0))] = 0.0
Rij[((0, 19), (2, 0))] = 0.0
Rij[((0, 19), (3, 0))] = 0.0
Rij[((0, 19), (4, 0))] = 0.0
Rij[((0, 19), (5, 0))] = 0.0
Rij[((0, 19), (6, 0))] = 0.0
Rij[((0, 19), (7, 0))] = 0.0
Rij[((0, 19), (8, 0))] = 0.0
Rij[((0, 19), (9, 0))] = 0.0
Rij[((0, 19), (10, 0))] = 0.0
Rij[((0, 19), (11, 0))] = 0.0
Rij[((0, 19), (12, 0))] = 0.0
Rij[((0, 19), (13, 0))] = 0.0
Rij[((0, 19), (14, 0))] = 0.0
Rij[((0, 19), (15, 0))] = 0.0
Rij[((0, 19), (16, 0))] = 0.0
Rij[((0, 19), (17, 0))] = 0.0
Rij[((0, 19), (18, 0))] = 0.0
Rij[((0, 19), (19, 0))] = 0.0
Rij[((1, 19), (0, 0))] = 0.0
Rij[((1, 19), (2, 0))] = 0.0
Rij[((1, 19), (3, 0))] = 0.0
Rij[((1, 19), (4, 0))] = 0.0
Rij[((1, 19), (5, 0))] = 0.0
Rij[((1, 19), (6, 0))] = 0.0
Rij[((1, 19), (7, 0))] = 0.0
Rij[((1, 19), (8, 0))] = 0.0
Rij[((1, 19), (9, 0))] = 0.0
Rij[((1, 19), (10, 0))] = 0.0
Rij[((1, 19), (11, 0))] = 0.0
Rij[((1, 19), (12, 0))] = 0.0
Rij[((1, 19), (13, 0))] = 0.0
Rij[((1, 19), (14, 0))] = 0.0
Rij[((1, 19), (15, 0))] = 0.

In [46]:
for i in location_idx:
    print(f"{i}: {Vit[(i,0)].varValue} == {Vit[(i,19)].varValue}")

0: 0.0 == 0.0
1: 0.0 == 0.0
2: 12.0 == 12.0
3: 0.0 == 0.0
4: 5.0 == 5.0
5: 0.0 == 0.0
6: 21.0 == 21.0
7: 0.0 == 0.0
8: 0.0 == 0.0
9: 8.0 == 8.0
10: 0.0 == 0.0
11: 0.0 == 0.0
12: 36.0 == 36.0
13: 10.0 == 10.0
14: 3.0 == 3.0
15: 0.0 == 0.0
16: 1.0 == 1.0
17: 0.0 == 0.0
18: 25.0 == 25.0
19: 3.0 == 3.0
